In [0]:
# Por si alguien corre en python2
from __future__ import division

# Preparamos todo para correr
import numpy as np
from math import *
from matplotlib import pyplot as plt
%matplotlib inline
from scipy.stats import norm, binom, gamma, poisson, multivariate_normal, uniform
from sklearn import linear_model

import pandas as pd
from pandas.plotting import scatter_matrix

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, confusion_matrix, accuracy_score, balanced_accuracy_score,precision_recall_curve, roc_curve, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, StratifiedShuffleSplit


import seaborn as sns

import os
import tarfile

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/exord/UNSAM_IA/master/"
HOUSING_PATH = os.path.join(".", "datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    #urllib.request.urlretrieve(housing_url, tgz_path)
    !wget http://raw.githubusercontent.com/exord/UNSAM_IA/master/datasets/housing.tgz -P {housing_path}
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

# Ejercicio: Regresion logistica multiclase

Utilizando el dataset de California, clasifique las distintas proximidades al oceano utilizando los features que prefiera (excepto longitud y latitud, para que no sea demasiado facil) y regresion logistica (utilice el paquete de scikit-learn).

In [0]:
fetch_housing_data()

housing = load_housing_data()
# housing.head()
print(type(housing))

Veamos que tengo 5 clases. En particular, island esta muy poco representada pero en general no estan balanceadas.

In [0]:
housing["ocean_proximity"].value_counts()

 La primer pregunta para hacerme es como quiero separar el dataset en train y test. Si lo hago estratificado, tendre muchos mas eventos de "$<$1H OCEAN" que de "ISLAND".

In [0]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=445543)
for train_index, test_index in split.split(housing, housing["ocean_proximity"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

print(strat_test_set["ocean_proximity"].value_counts() / len(strat_test_set))
print(housing["ocean_proximity"].value_counts()/len(housing))

Aca viene la primer decision grande. Que hago con la diferencia enorme entre clases?
 

1.   Si me interesa unicamente tener una gran cantidad de aciertos, puedo aplicar regresion logistica sin balancear.
2.   Si quiero poder predecir lo mejor posible cada clase, tengo que balancear los datos. Una implementacion puede hacerse utilizando la opcion "class_weight=’balanced’" en LogisticRegression.

Decidan ustedes! (yo hago ambas)

In [0]:
housing = strat_train_set.drop("ocean_proximity", axis=1) # drop labels for training set
housing_labels = strat_train_set["ocean_proximity"].copy()

cond = ~housing.isnull().any(axis=1)
housing_tr = housing[cond]
housing_labels = housing_labels[cond]

housing_num = housing_tr.drop(['latitude','longitude'], axis=1)
#housing_num = housing_tr[['latitude','longitude']]


scaler = StandardScaler()
hh = scaler.fit_transform(housing_num)

# Esto devuelve un array, que hay que transformar de nuevo a un DataFrame
housing_prepared = pd.DataFrame(hh, columns=housing_num.columns)
housing_prepared.head(10)

In [0]:
log_reg_balanced = LogisticRegression(max_iter=1000,class_weight='balanced')
log_reg_unbalanced = LogisticRegression(max_iter=1000)
log_reg_balanced.fit(housing_prepared, housing_labels)
log_reg_unbalanced.fit(housing_prepared, housing_labels)

Aca verifico que los shapes esten bien y que el output del regresor logistico sume 1.

In [0]:
print(housing_prepared.shape)
print(log_reg_balanced.predict_proba(housing_prepared).shape)
print(log_reg_balanced.predict_proba(housing_prepared).sum(axis=1))

Veamos un par de medidas de performance. En principio, la ya familiar matriz de confusion

In [0]:
print("Total per label")
print(confusion_matrix(housing_labels, log_reg_unbalanced.predict(housing_prepared)).sum(axis=1))
print("\nUnbalanced\n")
print(confusion_matrix(housing_labels, log_reg_unbalanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])
print("\nBalanced\n")
print(confusion_matrix(housing_labels, log_reg_balanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])

Y ahora veamos la accuracy.


In [0]:
print("Unbalanced\n")
print(accuracy_score(housing_labels, log_reg_unbalanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])
print("\nBalanced\n")
print(accuracy_score(housing_labels, log_reg_balanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])

Parece que balancear es peor, pero no, de la matriz de confusion sabemos que es distinto. Veamos el balanced_accuracy

In [0]:
print("Unbalanced\n")
print(balanced_accuracy_score(housing_labels, log_reg_unbalanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])
print("\nBalanced\n")
print(balanced_accuracy_score(housing_labels, log_reg_balanced.predict(housing_prepared)))#[[tn, fp],[fp, tp]])

Voy a evaluar la performance usando la ROC. En el caso multiclase hago una ROC para cada clase.

In [0]:
Y_unbalanced=log_reg_unbalanced.predict_proba(housing_prepared)
Y_balanced=log_reg_balanced.predict_proba(housing_prepared)

labels=np.asarray(['<1H OCEAN','INLAND','ISLAND','NEAR BAY','NEAR OCEAN'])

true_labels=np.zeros(Y_balanced.shape)
for i in range(len(labels)):
  true_labels[housing_labels==labels[i],i]=1.0

In [0]:
print(true_labels.sum(axis=0))

In [0]:
for i in range(len(labels)):
  fpr_balanced, tpr_balanced, thresholds_balanced = roc_curve(true_labels[:,i], Y_balanced[:,i])
  plt.plot(fpr_balanced,tpr_balanced, linestyle='dotted',color='blue',label='Balanced '+labels[i]+', AUC = '+str(round(roc_auc_score(true_labels[:,i], Y_balanced[:,i]),2)))
  indx_balanced=np.abs(thresholds_balanced-0.5*np.ones(len(thresholds_balanced)))<0.001
  plt.scatter(fpr_balanced[indx_balanced],tpr_balanced[indx_balanced],color='blue',label='Balanced '+labels[i]+' Decision point')
  fpr_unbalanced, tpr_unbalanced, thresholds_unbalanced = roc_curve(true_labels[:,i], Y_unbalanced[:,i])
  plt.plot(fpr_unbalanced,tpr_unbalanced, color='red',linestyle='dashed',label='Unbalanced '+labels[i]+', AUC = '+str(round(roc_auc_score(true_labels[:,i], Y_unbalanced[:,i]),2)))
  indx_unbalanced=np.abs(thresholds_unbalanced-0.5*np.ones(len(thresholds_unbalanced)))<0.005
  plt.scatter(fpr_unbalanced[indx_unbalanced],tpr_unbalanced[indx_unbalanced],color='red',label='Unbalanced '+labels[i]+' Decision point')
  plt.legend(loc='lower right')
  plt.show()

Ahora quedan un monton de cosas para hacer:


*   Pueden resolver esto mismo pero para solo dos clases.
*   Pueden utilizar unicamente latitud y longitud y ver que diferencias surgen entre balanceado e imbalanceado. Aca hay una ventaja: se puede hacer un plot de regiones de asignacion en el espacio de las variables. 
*   Pueden utilizar la distancia al oceano (y otras variables) para clasificar si la casa es comprable o no (poniendo algun threshold que defina si la casa es comprable o no). En particular, Para convertir distancia al oceano en una variable utilizable pruebe con el OneHotEncoder (ya vamos a ver mas al respecto luego para una situacion en la que sirva mas).
